# Merkle's Puzzles
In cryptography, Merkle's puzzles is an early construction for a public-key cryptosystem, a protocol devised by Ralph Merkle in 1974 and published in 1978. It allows two parties to agree on a shared secret by exchanging messages, even if they have no secrets in common beforehand.

## High Level Description
1. Bob generates 2^N messages containing, "This is message X. This is the symmetrical key, Y", where X is an identifier, and Y is a secret key meant for symmetrical encryption. Both X and Y are unique to each message. All the messages are encrypted in a way such that a user may conduct a brute force attack on each message with some difficulty. Bob sends all the encrypted messages to Alice.
1. Alice receives all the encrypted messages, and randomly chooses a single message to brute force. After Alice discovers both the identifier X and the secret key Y inside that message, she encrypts her clear text with the secret key Y, and sends that identifier (in cleartext) with her cipher text to Bob.
1. Bob finds the secret key paired with that identifier, and deciphers Alice's cipher text with that secret key.

## Implementation

Python implementation using [AES](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard) for symmetric encryption and secret module for [CSPRNG](https://en.wikipedia.org/wiki/Cryptographically_secure_pseudorandom_number_generator). The key used for the encryption is just a 256bit random number. Also for simplicity AES is used in [ECB](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation) mode which is not recommended for secure systems.

In [678]:
import secrets
from Crypto.Cipher import AES

Define some constants here. The puzzle difficulty is set to 8 bit which is very easy and in real system must be bigger.
The complexity of the entire puzzle depends also on the number of messages generated because an adversary could be able to solve all the puzzles and to find the PID sent by Alice to Bob.

In [679]:
PUZZLE_ID = b'AES Puzzle'
PUZZLE_BYTES = 16 # 128 bits
PUZZLE_DIFFICULTY = 8 # bits
MESSAGES_NUM = 2 ** 10
HASH_BYTES = 32 # 256 bits

In [680]:
keys = {}
puzzles = []

## Functions

In [681]:
"""
This function pads a byte array to length of *16
"""
def pad16(b):
    n = 16-len(b) % 16;
    r = secrets.randbits(n*8).to_bytes(n, 'big')
    return b + r

In [682]:
"""
Creates a puzzle using AES cipher. Generates a random PUZZLE_DIFFICULTY bits which is filled up to PUZZLE_BYTES with zeros.
Then PK is generated which will be the private key for the communication which only Alice and Bob will know. PID is generated
to be the index of PK.
"""
def createPuzzle():
    key = secrets.randbits(PUZZLE_DIFFICULTY).to_bytes(PUZZLE_BYTES, 'big')
    cipher = AES.new(key, AES.MODE_ECB)
    pid = secrets.randbits(HASH_BYTES*8).to_bytes(HASH_BYTES, 'big')
    pk = secrets.randbits(HASH_BYTES*8).to_bytes(HASH_BYTES, 'big')
    msg = PUZZLE_ID + pid + pk
    puzzle = cipher.encrypt(pad16(msg))
    
    return puzzle, pid, pk

In [683]:
"""
Attempt to brute force a puzzle. Returns PID and PK, as well as bool if successful
"""
def bruteForce(puzzle):
    for i in range(0, 2 ** PUZZLE_DIFFICULTY):        
        key = i.to_bytes(PUZZLE_BYTES, 'big')        
        cipher = AES.new(key, AES.MODE_ECB)
        msg = cipher.decrypt(puzzle)
        if msg.find(PUZZLE_ID) == 0:
            offset = len(PUZZLE_ID)
            pid = msg[offset:offset+HASH_BYTES]
            pk = msg[offset+HASH_BYTES:offset+HASH_BYTES*2]
            return True, pid, pk
    return False

In [684]:
"""
Encrypts a message using private key PK. PID is appended to the message to be used as index from the other party.
Message length is also included since the padding changes the original message
"""
def encryptMessage(pid, pk, msg):
    cipher = AES.new(pk, AES.MODE_ECB)
    msg_len = len(msg).to_bytes(4, 'big')
    return pid + msg_len + cipher.encrypt(pad16(msg))

In [685]:
"""
Decrypts a message. The message format must be PID + MESSAGE_LEN + AES(message)
"""
def descryptMessage(secret):
    pid = secret[:HASH_BYTES]
    msg_len = int.from_bytes(secret[HASH_BYTES:HASH_BYTES+4], 'big')
    msg = secret[HASH_BYTES+4:]
    pk = keys[pid] 
    cipher = AES.new(pk, AES.MODE_ECB)
    decrypted = cipher.decrypt(msg)
    return decrypted[:msg_len]

## Example

In [686]:
# 1. Bob generates 2^N messages 
for i in range(0, MESSAGES_NUM):
    puzzle, pid, pk = createPuzzle()
    keys[pid] = pk
    puzzles.append(puzzle)
'Bob has generated {} puzzles'.format(len(puzzles))

'Bob has generated 1024 puzzles'

In [687]:
# 2. Alice receives all the encrypted messages, and randomly chooses a single message to brute force
try_puzzle = secrets.choice(puzzles)
'Alice has chosen a puzzle: ' + try_puzzle.hex()


'Alice has chosen a puzzle: b4a751ec490fddb3b3c06f3e73c2aefb879c641b8c487c78fec07b57c91a61436bfb276d097def24335bd59bececc5a9e81fbd2503637280ba1d80f6799f3fbd216de3e42ce5f76a2fe9ede74388366f'

In [688]:
cracked, pid, pk = bruteForce(try_puzzle)
if not cracked:
    raise Error('Hmm, something went wrong')

'Alice has cracked the puzzle and found: PID:{} and PK:{}'.format(pid.hex(), pk.hex())

'Alice has cracked the puzzle and found: PID:335cf2011426a1cdf7075f15a65b927a87cfe7ed107f0f0ab065e750434630c1 and PK:54253f2951d490cc11d975c6fa7710048a2b52e0f2f6cb33c656342bce94e132'

In [689]:
secret = encryptMessage(pid, pk, b'Hi Bob, this is secure channel')
'Alice has sent encrypted message to Bob sayng "Hi Bob, this is secure channel"'

'Alice has sent encrypted message to Bob sayng "Hi Bob, this is secure channel"'

In [734]:
# 3. Bob finds the secret key paired with that identifier, and deciphers Alice's cipher text with that secret key.
decrypted = descryptMessage(secret)
'Bob has decrypted the message and it was: "{}"'.format(str(decrypted, 'utf-8'))

'Bob has decrypted the message and it was: "Hi Bob, this is secure channel"'

## References

1. Wikipedia, "Merkle's Puzzles", https://en.wikipedia.org/wiki/Merkle%27s_Puzzles